# Sharpness distribution across image

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import h5py
#To import DanMAX from the folder above:
import sys
sys.path.append('../')
import DanMAX as DM
style = DM.darkMode(style_dic={'figure.figsize':'large'})

In [ ]:
fname = DM.findScan().replace('_orca', '') #

In [ ]:
print(fname)
fh = h5py.File(fname, 'r')
motname = DM.getScanType(fname).split()[1]
images = fh['/entry/instrument/orca/data'][:]
motor = fh['/entry/instrument/'+motname+'/value'][:]

In [ ]:
def calc_sharpness(image):
    gx, gy = np.gradient(image)
    gnorm = np.sqrt(gx**2 + gy**2)
    shrp = np.average(gnorm)
    shrp_x = np.average(np.abs(gx))
    shrp_y = np.average(np.abs(gy))
    return shrp, shrp_x, shrp_y

def flatten_image(image):
    result = image/(np.mean(image, axis=1))[:,None]
    result = image/(np.mean(result, axis=0))[None,:]
    return result

In [ ]:
sharpness = []
sharpness_x = []
sharpness_y = []
sharpness_tl = []
sharpness_tr = []
sharpness_bl = []
sharpness_br = []

for i in images:
    flat = flatten_image(i)
    res = calc_sharpness(flat)
    sharpness.append(res[0])
    sharpness_x.append(res[1])
    sharpness_y.append(res[2])
    flat_tl = flatten_image(i)[-300:,0:300]
    res_tl = calc_sharpness(flat_tl)
    sharpness_tl.append(res_tl[0])
    flat_tr = flatten_image(i)[-300:,-300:]
    res_tr = calc_sharpness(flat_tr)
    sharpness_tr.append(res_tr[0])
    flat_bl = flatten_image(i)[0:300:,0:300]
    res_bl = calc_sharpness(flat_bl)
    sharpness_bl.append(res_bl[0])
    flat_br = flatten_image(i)[0:300,-300:]
    res_br = calc_sharpness(flat_br)
    sharpness_br.append(res_br[0])

argm = np.argmax(sharpness)

motmax = motor[argm]

In [ ]:
plt.figure()
plt.plot(motor, (sharpness - np.min(sharpness))/(np.max(sharpness)-np.min(sharpness)), label='total')
plt.plot(motor, (sharpness_x - np.min(sharpness_x))/(np.max(sharpness_x)-np.min(sharpness_x)), label='horizontal')
plt.plot(motor, (sharpness_y - np.min(sharpness_y))/(np.max(sharpness_y)-np.min(sharpness_y)), label='vertical')
plt.legend()
plt.show()

bestim = flatten_image(images[argm])
print('Best focus found at {} (step # {})'.format(motor[argm], argm))
print('Recommended motion command')
print('umv {} {:.4f}'.format(str(motname), motor[argm]))

In [ ]:
plt.figure()
plt.plot(motor, (sharpness_tl - np.min(sharpness_tl))/(np.max(sharpness_tl)-np.min(sharpness_tl)), label='top left')
plt.plot(motor, (sharpness_tr - np.min(sharpness_tr))/(np.max(sharpness_tr)-np.min(sharpness_tr)), label='top right')
plt.plot(motor, (sharpness_bl - np.min(sharpness_bl))/(np.max(sharpness_bl)-np.min(sharpness_bl)), label='bottom left')
plt.plot(motor, (sharpness_br - np.min(sharpness_br))/(np.max(sharpness_br)-np.min(sharpness_br)), label='bottom right')
plt.legend()
plt.show()

bestim = flatten_image(images[argm])
print('Best focus found at {} (step # {})'.format(motor[argm], argm))
print('Recommended motion command')
print('umv {} {:.4f}'.format(str(motname), motor[argm]))

In [ ]:
fullresim = np.zeros(shape=(bestim.shape[0]//100, bestim.shape[1]//100))

for y in range (0, fullresim.shape[0]):
    for x in range (0, fullresim.shape[1]):
        fracim = bestim[y*100:(y+1)*100,x*100:(x+1)*100]
        fullresim[y,x] = calc_sharpness(fracim/np.mean(fracim))[0]
        #print(fullresim)

In [ ]:
print('sharpness distribution across the image')
plt.figure()
plt.imshow(fullresim)